This notebook will test the effectiveness of the Auxiliary GAN method for generating synthetic seizure data. It will be assessed using the TSTR method.

Notes/To do:
- Fix the feature_net to make it output the right shape. Compare it to the SuperGAN one

In [1]:
import numpy as np
from tensorflow.keras.models import Model, load_model
from sklearn.metrics import accuracy_score
import training_module as train
import conditional_training_module as cond_train
import input_module as input_mod
import saving_module as save
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import sys, models, conditional_models, tensorflow.keras

DATAFILE = "/Users/nburley/gans_deep_learning/IMWUT_GAN/code/NateBurley_Research_GANs/data/Epileptic_Seizure_Recognition.csv"
NUM_OG_CLASSES = 5
num_classes = 2

# Global variables for training
NUM_CLASSES = 2
NUM_CLASSIFIER_EPOCHS = 12
NUM_TSTR_CLASSIFIER_EPOCHS = 20
NUM_GAN_EPOCHS = 40 # Formerly 25. Curious if RTS or SFD is more important
CLASSIFIER_TRAIN_RATIO = 0.8
NUM_SYNTHETIC_SAMPLES = 2000 # Per class
NUM_STATS = 9 # Number of statistics computed for loss function. Do NOT change

# PARAMETERS RELATED TO TRAINING
latent_dim = 70 #length of random input fed to generator (should it be num_classes + 1?)
epochs = 100 #num training epochs
batch_size = 30 #num instances generated for G/D training
test_size = 100 #num instances generated for validating data
real_synthetic_ratio = 5 #num synthetic instances per real instance for computing RTS metric
synthetic_synthetic_ratio = 10 #num synthetic instances to compare for computing STS metric
disc_lr = .08 #learning rate of discriminator
accuracy_threshold = 0.95 #threshold to stop generator training
instances_per_class_train = 10
instances_per_class_test = 10

# WEIGHTS FOR DIFFERENT TERMS IN THE LOSS FUNCTION
D_loss_weight = 1.25
D_loss_weight2 = 1.25
C_loss_weight = 0.75
SFD_loss_weight = 1

Read the data into pandas dataframes:

In [2]:
# Read in the data; seperate by class
eeg_raw = pd.read_csv(DATAFILE)
class_df_dict = {}
for y_val, eeg_df in eeg_raw.groupby('y'):
    eeg_df.columns = [int(i) for i in range(1, (len(eeg_df.columns)+1))]
    eeg_df.rename(columns={ eeg_df.columns[len(eeg_df.columns)-1]: "y" }, inplace=True)
    class_df_dict[y_val] = eeg_df

# Join non-seizure classes into one dataset
#X_0_df = pd.concat([class_df_dict[x] for x in range(2, NUM_OG_CLASSES+1)])
X_0_df = class_df_dict[2]
X_0_df.loc[:,'y'] = 0
Y_0_df = X_0_df['y']
del X_0_df['y']
X_0_df = X_0_df.apply(pd.to_numeric)
X_0_df = X_0_df.sample(frac=1) # Shuffle rows

# Normalize
X_0_df = X_0_df / 1000
print(X_0_df.head(5))

# Build seizure dataset
X_1_df = class_df_dict[1]
X_1_df.loc[:,'y'] = 1
Y_1_df = X_1_df['y']
del X_1_df['y']
X_1_df = X_1_df.apply(pd.to_numeric)
X_0_df = X_0_df.sample(frac=1) # Shuffle rows

# Normalize
X_1_df = X_1_df / 1000
print(X_1_df.head(5))

         1      2      3      4      5      6      7      8      9      10   \
4748   0.008  0.027  0.029  0.009 -0.032 -0.058 -0.075 -0.080 -0.079 -0.105   
468   -0.003 -0.014 -0.022 -0.027 -0.031 -0.033 -0.030 -0.024 -0.019 -0.009   
10289 -0.183 -0.232 -0.272 -0.315 -0.341 -0.378 -0.396 -0.425 -0.443 -0.469   
7262  -0.057 -0.061 -0.054 -0.050 -0.051 -0.051 -0.066 -0.076 -0.081 -0.082   
7504  -0.003 -0.008 -0.015 -0.023 -0.025 -0.023 -0.023 -0.017 -0.020 -0.023   

       ...    169    170    171    172    173    174    175    176    177  \
4748   ... -0.022 -0.005 -0.007 -0.018 -0.031 -0.041 -0.053 -0.052 -0.045   
468    ... -0.001 -0.002 -0.004 -0.006 -0.006 -0.005 -0.008 -0.014 -0.014   
10289  ...  0.037  0.044  0.059  0.066  0.076  0.079  0.076  0.061  0.034   
7262   ... -0.050 -0.039 -0.036 -0.035 -0.036 -0.033 -0.024 -0.014 -0.004   
7504   ...  0.008 -0.005 -0.015 -0.020 -0.020 -0.018 -0.023 -0.022 -0.023   

         178  
4748  -0.036  
468   -0.014  
10289  0.009  
72

/Users/nburley/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


Next, we convert the data to numpy arrays:

In [3]:
# Join them together
Y_df = pd.concat((Y_0_df, Y_1_df))
X_df = pd.concat((X_0_df, X_1_df))

# Build X_0 and Y_0 datasets
Y_0 = Y_0_df.values.T
Y_0 = Y_0.reshape((Y_0.shape[0], 1))
X_0 = X_0_df.values
#X_0 = normalize(X_0)
X_0 = X_0.reshape((X_0.shape[0], X_0.shape[1], 1))

Y_1 = Y_1_df.values.T
Y_1 = Y_1.reshape((Y_1.shape[0], 1))
X_1 = X_1_df.values
#X_1 = normalize(X_1)
X_1 = X_1.reshape((X_1.shape[0], X_1.shape[1], 1))

# Convert to numpy arrays, reshape
Y = Y_df.values.T
Y = Y.reshape((Y.shape[0], 1))
X = X_df.values
#X = normalize(X)
X = X.reshape((X.shape[0], X.shape[1], 1))
print("Y shape: {}".format(Y.shape))
print("X shape: {}".format(X.shape))

# Build training and testing sets
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, \
    train_size=CLASSIFIER_TRAIN_RATIO, shuffle=True)
print("\nTrain X shape: {}".format(train_X.shape))
print("Test X shape: {}".format(test_X.shape))

# Convert the Y's to categorical
Y = tensorflow.keras.utils.to_categorical(Y)
Y_0 = tensorflow.keras.utils.to_categorical(Y_0)
#Y_1 = keras.utils.np_utils.to_categorical(Y_1)
train_Y = tensorflow.keras.utils.to_categorical(train_Y)
test_Y = tensorflow.keras.utils.to_categorical(test_Y)
print("Train Y shape: {}".format(train_Y.shape))
print("Test Y shape: {}".format(test_Y.shape))
print("\nY_0 shape: {}".format(Y_0.shape))
print("Y_1 shape: {}".format(Y_1.shape))
print("X_0 shape: {}".format(X_0.shape))
print("X_1 shape: {}".format(X_1.shape))

# Parameters for training shape
num_seqs = X.shape[0]
seq_length = X.shape[1]
num_channels = X.shape[2]
input_shape = (seq_length, num_channels)
print("\nIn shape: {}".format(input_shape))

Y shape: (4600, 1)
X shape: (4600, 178, 1)

Train X shape: (3680, 178, 1)
Test X shape: (920, 178, 1)
Train Y shape: (3680, 2)
Test Y shape: (920, 2)

Y_0 shape: (2300, 1)
Y_1 shape: (2300, 1)
X_0 shape: (2300, 178, 1)
X_1 shape: (2300, 178, 1)

In shape: (178, 1)


Next, we define the generator model (since this architecture apparently doesn't use a classifier?), and then train the model (because who gives a fuck idc to split this up):

In [4]:
#CREATE VECTOR OF LABELS USED FOR TRAINING C AND CONDITIONING G AND D
class_labels_target_train = cond_train.create_target_label_vector_all_classes(instances_per_class_train, seq_length, num_classes)[0] #these are one-hot since they're fed to keras model
class_labels_target_test = cond_train.create_target_label_vector_all_classes(instances_per_class_test, seq_length, num_classes)[1] #these are standard since sklearn evaluation methods are used
class_labels_input_train = cond_train.create_input_label_vector_all_classes(instances_per_class_train, seq_length, num_classes)

#CREATE GENERATOR AND DISCRIMINATOR
G = conditional_models.create_cond_G(seq_length, num_channels, NUM_CLASSES, latent_dim)
D = conditional_models.create_AC_D(seq_length, num_channels, NUM_CLASSES)
D_to_freeze = D
D_model = models.compile_discriminator_model(D, disc_lr)

#CREATE STATISTICAL FEATURE NETWORK AND COMPUTE FEATURE VECTOR FOR REAL DATA (used in loss function)
feature_net = models.create_statistical_feature_net(seq_length, num_channels)
S_X_train, S_X_test = cond_train.compute_statistical_vector(X,Y,feature_net,num_channels,NUM_CLASSES,instances_per_class_train,instances_per_class_test)

#CREATE FULL ARCHITECTURE WHERE OUPUT OF GENERATOR IS FED TO DISCRIMINATOR AND CLASSIFIER
for layer in D_to_freeze.layers:
    layer.trainable = False
GCD = Model(inputs=G.input, outputs=[D_to_freeze(G.output[1])[0], D_to_freeze(G.output[1])[1],feature_net(G.output[0])])
GCD.compile(loss=["binary_crossentropy","categorical_crossentropy", train.euc_dist_loss], 
            optimizer="adam", metrics={"D":"accuracy"},
            loss_weights = [D_loss_weight, D_loss_weight2 ,SFD_loss_weight])

'''
GCD.compile(loss={"D":"binary_crossentropy","C":"categorical_crossentropy","SFN": train.euc_dist_loss}, 
			optimizer="adam", metrics={"D":"accuracy",'C':"accuracy"},
			loss_weights = {"D": D_loss_weight, "C": C_loss_weight,"SFN": SFD_loss_weight})
'''


GC_acc=0
epoch=1
max_RTS = 0
max_RTS_epoch = 0
min_STS = 1000
min_STS_epoch = 0
min_SFD = 10000
min_SFD_epoch = 0
while (GC_acc<accuracy_threshold) and (epoch <= 100):
    print("Epoch: " + str(epoch))

    #TRAIN DISCRIMINATOR AND GENERATOR AND DISPLAY ACCURACY FOR EACH
    D_loss_vec = cond_train.train_AC_D(instances_per_class_train, X, Y, class_labels_input_train, class_labels_target_train, NUM_CLASSES, G, D_model, seq_length, latent_dim)
    GCD_loss_vec = cond_train.train_AC_G(instances_per_class_train, X, class_labels_input_train, class_labels_target_train, S_X_train, NUM_CLASSES, GCD, seq_length, latent_dim)
    D_acc = D_loss_vec[1] #accuracy for discriminator during its "turn" for training
    GD_acc = GCD_loss_vec[4] #accuracy for generator in tricking discriminator
    print("D Acc: " + str(D_acc))
    print("G Acc in tricking D: " + str(GD_acc))

    #GENERATE SYNTHETIC DATA AND FEED TO CLASSIFIER TO DETERMINE ACCURACY
    synthetic_data = cond_train.generate_synthetic_data_all_classes(test_size, G, class_label, NUM_CLASSES, latent_dim, seq_length)
#     pred = C.predict_classes(synthetic_data,test_size,verbose=0)
#     #true = [class_label]*test_size
#     GC_acc = accuracy_score(class_labels_target_test, pred)
#     print("C acc for synthetic data: " + str(GC_acc))

    #COMPUTE RTS AND STS METRICS
    mean_RTS_sim, mean_STS_sim, _ = train.compute_similarity_metrics(synthetic_data, X, test_size,real_synthetic_ratio, synthetic_synthetic_ratio)
    print("RTS similarity: " + str(mean_RTS_sim))
    print("STS similarity: " + str(mean_STS_sim))

    #COMPUTE STATISTICAL FEATURE DISTANCE
    synthetic_features = feature_net.predict(synthetic_data, test_size, verbose=0)
    SFD = train.compute_SFD(synthetic_features, S_X_test)
    print("SFD: " + str(SFD))

    #IF DESIRED, SAVE GENERTOR MODEL / WRITE TRAINING RESULTS
    if model_save_directory!=False:
        save.save_G(G, epoch, class_label, model_save_directory)
    if write_train_results == True:
        save.write_results(outfile, epoch, class_label, D_acc, GD_acc, GC_acc, mean_RTS_sim, mean_STS_sim)
    
    #RE-EVALUATE OUR TALLIES OF EPOCHS WITH LOWEST STS, HIGHEST RTS
    if (mean_RTS_sim > max_RTS) and epoch >= 25:
        max_RTS_epoch = epoch
        max_RTS = mean_RTS_sim
    if (mean_STS_sim < min_STS) and epoch >= 25:
        min_STS_epoch = epoch
        min_STS = mean_STS_sim
    if (SFD < min_SFD) and epoch >= 25:
        min_SFD_epoch = epoch
        min_SFD = SFD
        print("Updated SFD")

    epoch+=1
    one_segment_real = np.reshape(X[np.random.randint(0, X.shape[0], 1)], (seq_length, num_channels))

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 178, 3)]     0                                            
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 178, 3)       0           input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 100)          41600       dropout_2[0][0]                  
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1)            101         lstm_1[0][0]                     
____________________________________________________________________________________________

AxisError: axis 2 is out of bounds for array of dimension 2